## Import Libaries

In [0]:
import pandas as pd
import numpy as np
import os
# Torch and Torchvision
import torchvision 
#from torch.utils import data
from torch.utils.data import Dataset, TensorDataset,DataLoader
from torchvision import datasets
import torchvision.transforms as transforms 
import matplotlib.pyplot as plt
from PIL import Image
import torch

In [0]:
!pip install tensorboardX
import tensorflow as tf
import numpy as np
import scipy.misc
from tensorboardX import SummaryWriter
%load_ext tensorboard
import os
logs_base_dir = "Logs"
os.makedirs(logs_base_dir, exist_ok=True)
# debug ref: https://github.com/pytorch/pytorch/issues/22676

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
import seaborn as sns
# PyTorch
from torchvision import transforms, datasets, models
import torch
from torch import optim, cuda
from torch.utils.data import DataLoader, sampler
import torch.nn as nn

#import warnings
#warnings.filterwarnings('ignore', category=FutureWarning)

# Data science tools
import numpy as np
import pandas as pd
import os

# Image manipulations
from PIL import Image
# Useful for examining network
from torchsummary import summary
# Timing utility
from timeit import default_timer as timer

# Visualizations
import matplotlib.pyplot as plt
#%matplotlib inline
plt.rcParams['font.size'] = 14

## Data Preparation

### Load Data and check Meta info

In [0]:
transforms_raw = transforms.Compose([transforms.ToTensor()]) #for conversion to tensors
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transforms_raw)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transforms_raw)

Extracting ./data/cifar-10-python.tar.gz to ./data

Files already downloaded and verified


In [0]:
# get train, test set sizes
trainset_size =len(trainset) 
testset_size = len(testset)
print(trainset_size, testset_size)

50000 10000


In [0]:
#trainset.targets

In [0]:
# meta data information
trainset.class_to_idx

{'airplane': 0,
 'automobile': 1,
 'bird': 2,
 'cat': 3,
 'deer': 4,
 'dog': 5,
 'frog': 6,
 'horse': 7,
 'ship': 8,
 'truck': 9}

In [0]:
# Calculate mean and std of the pixel values in images and nomalize it while transforming
x = np.concatenate([np.asarray(
    trainset[i][0].reshape(-1,
                             trainset[0][0].shape[0],
                             trainset[0][0].shape[1],
                             trainset[0][0].shape[2])
    ) for i in range(len(trainset))])
# print(x)
print(x.shape)

(50000, 3, 32, 32)


In [0]:
#trainset.data[0]
#trainset[0][0]

In [0]:
# calculate the mean and std along the (0, 1) axes
train_mean = np.mean(x, axis=(0,2,3))
train_std = np.std(x, axis=(0,2,3))
print(train_mean)
print(train_std)

[0.4914009  0.48215896 0.4465308 ]
[0.24703279 0.24348423 0.26158753]


### Filter 50% of the labels for training (bird, deer and truck)

In [0]:
# traning data labels for bird = 2, deer =4, truck = 9 need to be cut down to half 
custom_data=[]
custom_label =[]
bird = 0
deer =0
truck = 0
limit = 2500
for i in range(len(trainset)):#range(20):
    train_data_i = trainset[i][0]
    train_label_i = trainset[i][1]
    rules = [train_label_i == 0,
             train_label_i == 1,
             train_label_i == 3,
             train_label_i == 5,
             train_label_i == 6,
             train_label_i == 7,
             train_label_i == 8]
    if any(rules):
        custom_data.append(train_data_i)
        custom_label.append(train_label_i)      
    elif train_label_i==2:
      if bird<limit:
        bird+=1
        custom_data.append(train_data_i)
        custom_label.append(train_label_i)
    elif train_label_i==4:
      if deer<limit:
        deer+=1
        custom_data.append(train_data_i)
        custom_label.append(train_label_i)
    elif train_label_i==9:
      if truck<limit:
        truck+=1
        custom_data.append(train_data_i)
        custom_label.append(train_label_i)        


In [0]:
print(len(custom_data))
#tensor_clabel = torch.Tensor(custom_label)
#tensor_cdata = torch.Tensor(custom_data)

42500


In [0]:
print(len(custom_data))
tensor_clabel = custom_label
tensor_cdata = custom_data

42500


In [0]:
#df = pd.DataFrame({'img_data':custom_data,'class_label':custom_label})
#df.iloc[1]['img_data'].shape
#df.to_csv('./data/xx.csv')
#ss = pd.read_csv('./data/xx.csv')

### Build Customed Dataset with Data Augumentation

In [0]:
# my_dataset = TensorDataset(tensor_cdata,tensor_clabel)
# ref:https://stackoverflow.com/questions/44429199/how-to-load-a-list-of-numpy-arrays-to-pytorch-dataset-loader
# By default transforms are not supported for TensorDataset.
# but we want to use the the horizonalflip etc as data augumentation methods 
# Threfore we will create our custom class to add that option.
# ref: https://stackoverflow.com/questions/55588201/pytorch-transforms-on-tensordataset
class CustomTensorDataset():
    """TensorDataset with support of transforms.
    """
    def __init__(self, images,labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __getitem__(self, index):
        labels = self.labels[index]
        image_tensor = self.images[index]
        images = transforms.ToPILImage()(image_tensor)

        if self.transform:
            images = self.transform(images)


        return images, labels

    def __len__(self):
        return len(self.labels)

In [0]:
train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize(train_mean, train_std)
                                     ])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize(train_mean, train_std)
                                    ])

In [0]:
# we will use a rough 80% train, 10% val, 10% test spilt 
cifar10datasets = {
    'train':CustomTensorDataset(images=tensor_cdata,labels=tensor_clabel,
     transform=train_transform)
            }
valtest_dataset= datasets.CIFAR10(
         root='./data', train=False, download=True, 
         transform=test_transform)
cifar10datasets['test'], cifar10datasets['val'] = torch.utils.data.random_split(valtest_dataset, [5000, 5000])

Files already downloaded and verified


### Build  Data Iterators

 

To avoid loading all of the data into memory at once, we use training `DataLoaders`. First, we create a dataset object from the image folders, and then we pass these to a `DataLoader`. At training time, the `DataLoader` will load the images from disk, apply the transformations, and yield a batch. To train and validation, we'll iterate through all the batches in the respective `DataLoader`. 

One crucial aspect is to `shuffle` the data before passing it to the network. This means that the ordering of the image categories changes on each pass through the data (one pass through the data is one training epoch).

In [0]:
batch_size=16
dataloaders = {
'train' : DataLoader(cifar10datasets['train'], batch_size=batch_size,shuffle=True, num_workers=2),
'test' : DataLoader(cifar10datasets['test'], batch_size=batch_size,shuffle=True, num_workers=2),
'val' : DataLoader(cifar10datasets['val'], batch_size=batch_size,shuffle=True, num_workers=2)}

## Load Prtrained Autoencoder

In [0]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()

        # Input size to the Autoencoder : [batch_size, 3, 32, 32]
        # Output size from the Autoencoder : [batch_size, 3, 32, 32]
        self.encoder = nn.Sequential(
            # 3 input channels(RGB), 12 output channels, 4x4 square convolution
            nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 4, stride=2, padding=1), 
            nn.ReLU(),# Output size : [batch_size, 12, 32, 32]
            nn.Conv2d(in_channels = 12, out_channels = 24, kernel_size = 4, stride=2, padding=1), 
            nn.ReLU(),# Output size : [batch_size, 24, 32, 32]
            nn.Conv2d(in_channels = 24, out_channels = 48, kernel_size = 4, stride=2, padding=1),
            nn.ReLU() 
            ## Output size : [batch_size, 48, 4, 4]
        )
        self.decoder = nn.Sequential(
          
            nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1), 
            nn.ReLU(),  #TODO : try with LeakyReLU while decoding
            nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1), # Output size: [batch_size, 12, 32, 32]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1), # Output size: [batch_size, 3, 32, 32]
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [0]:

def create_AEmodel():
    autoencoder = Autoencoder()
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU.")
    return autoencoder

In [0]:
# Create model
autoencoder = create_AEmodel()

Model moved to GPU.


In [0]:
# load pretrained weights
autoencoder.load_state_dict(torch.load('./autoencoder.pkl'))

<All keys matched successfully>

In [0]:
for param in autoencoder.parameters():
    param.requires_grad = False

In [0]:
summary(autoencoder, input_size=(3, 32, 32), batch_size=batch_size, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [16, 12, 16, 16]             588
              ReLU-2           [16, 12, 16, 16]               0
            Conv2d-3             [16, 24, 8, 8]           4,632
              ReLU-4             [16, 24, 8, 8]               0
            Conv2d-5             [16, 48, 4, 4]          18,480
              ReLU-6             [16, 48, 4, 4]               0
   ConvTranspose2d-7             [16, 24, 8, 8]          18,456
              ReLU-8             [16, 24, 8, 8]               0
   ConvTranspose2d-9           [16, 12, 16, 16]           4,620
             ReLU-10           [16, 12, 16, 16]               0
  ConvTranspose2d-11            [16, 3, 32, 32]             579
          Sigmoid-12            [16, 3, 32, 32]               0
Total params: 47,355
Trainable params: 0
Non-trainable params: 47,355
---------------------------------

## Simple Classification model

In [0]:
class CNNClassifier(nn.Module):

    def __init__(self):
        super(CNNClassifier, self).__init__()

        self.conv_layer = nn.Sequential(

            # Conv Layer block 1
            nn.Conv2d(in_channels=48, out_channels=48, kernel_size=3, padding=1),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=48, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            # Conv Layer block 2
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.05)
            
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(128, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 10)
        )


    def forward(self, x):
        
        # conv layers
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)

        return x

## Pretrained Model

In [0]:
#model = pretrainedmodel()
model = CNNClassifier()
if torch.cuda.is_available():
    model = model.cuda()
summary(model, input_size=(48, 4, 4), batch_size=16, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [16, 48, 4, 4]          20,784
       BatchNorm2d-2             [16, 48, 4, 4]              96
              ReLU-3             [16, 48, 4, 4]               0
            Conv2d-4             [16, 32, 4, 4]          13,856
              ReLU-5             [16, 32, 4, 4]               0
         MaxPool2d-6             [16, 32, 2, 2]               0
            Conv2d-7             [16, 32, 2, 2]           9,248
       BatchNorm2d-8             [16, 32, 2, 2]              64
              ReLU-9             [16, 32, 2, 2]               0
        Dropout2d-10             [16, 32, 2, 2]               0
          Dropout-11                  [16, 128]               0
           Linear-12                   [16, 64]           8,256
             ReLU-13                   [16, 64]               0
           Linear-14                   

#### Mapping of Classes to Indexes

To keep track of the predictions made by the model, we create a mapping of classes to indexes and indexes to classes. This will let us know the actual class for a given prediction.

In [0]:
model.class_to_idx = trainset.class_to_idx
model.idx_to_class = {
    idx: class_
    for class_, idx in model.class_to_idx.items()
}

list(model.idx_to_class.items())

[(0, 'airplane'),
 (1, 'automobile'),
 (2, 'bird'),
 (3, 'cat'),
 (4, 'deer'),
 (5, 'dog'),
 (6, 'frog'),
 (7, 'horse'),
 (8, 'ship'),
 (9, 'truck')]

# Training Loss and Optimizer

The loss is the negative log likelihood and the optimizer is the Adam optimizer. The negative log likelihood in PyTorch expects log probabilities so we need to pass it the raw output from the log softmax in our model's final layer. The optimizer is told to optimizer the model parameters (only a few of which require a gradient). 

* Loss (criterion): keeps track of the loss itself and the gradients of the loss with respect to the model parameters (weights)
* Optimizer: updates the parameters (weights) with the gradients 

In [0]:
criterion = nn.CrossEntropyLoss() #nn.NLLLoss()
# NLLLoss  is particularly useful when you have an unbalanced training set.
# https://discuss.pytorch.org/t/difference-between-cross-entropy-loss-or-log-likelihood-loss/38816
optimizer = optim.Adam(model.parameters())

Below we can look at the parameters (weights) that will be updated by the optimizer during training.

In [0]:
for p in optimizer.param_groups[0]['params']:
    if p.requires_grad:
        print(p.shape)

torch.Size([48, 48, 3, 3])
torch.Size([48])
torch.Size([48])
torch.Size([48])
torch.Size([32, 48, 3, 3])
torch.Size([32])
torch.Size([32, 32, 3, 3])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([64, 128])
torch.Size([64])
torch.Size([32, 64])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


# Training

For training, we iterate through the train `DataLoader`, each time passing one batch through the model. One complete pass through the training data is known as an `epoch`, and we train for a set number of epochs or until early stopping kicks in (more below). After each batch, we calculate the loss (with `criterion(output, targets)`) and then calculate the gradients of the loss with respect to the model parameters with `loss.backward()`. This uses autodifferentiation and backpropagation to calculate the gradients. 

After calculating the gradients, we call `optimizer.step()` to update the model parameters with the gradients. This is done on every training batch so we are implementing stochastic gradient descent (or rather a version of it with momentum known as Adam). For each batch, we also compute the accuracy for monitoring and after the training loop has completed, we start the validation loop. This will be used to carry out early stopping.


## Early Stopping

Early stopping halts the training when the validation loss has not decreased for a number of epochs. Each time the validation loss does decrease, the model weights are saved so we can later load in the best model. Early stopping is an effective method to prevent overfitting on the training data. If we continue training, the training loss will continue to decrease, but the validation loss will increase because the model is starting to memorize the training data. Early stopping prevents this from happening, and, if we save the model each epoch when the validation loss decreases, we are able to retrieve the model that does best on the validation data.

Early stopping is implemented by iterating through the validation data at the end of each training epoch and calculating the loss. We use the complete validation data every time and record whether or not the loss has decreased. If it has not for a number of epochs, we stop training, retrieve the best weights, and return them. When in the validation loop, we make sure not to update the model parameters. 

### Training Function

The below function trains the network while monitoring a number of different parameters. We train with early stopping on the validation set. There are a number of parameters that I've tried to explain in the doc string. Hopefully, the comments and background make things somewhat understandable! 

In [0]:
def train(model,
          autoencoder,
          criterion,
          optimizer,
          train_loader,
          valid_loader,
          save_file_name,
          max_epochs_stop=3,
          n_epochs=20,
          print_every=2):
    """Train a PyTorch Model

    Params
    --------
        model (PyTorch model): cnn to train
        criterion (PyTorch loss): objective to minimize
        optimizer (PyTorch optimizier): optimizer to compute gradients of model parameters
        train_loader (PyTorch dataloader): training dataloader to iterate through
        valid_loader (PyTorch dataloader): validation dataloader used for early stopping
        save_file_name (str ending in '.pt'): file path to save the model state dict
        max_epochs_stop (int): maximum number of epochs with no improvement in validation loss for early stopping
        n_epochs (int): maximum number of training epochs
        print_every (int): frequency of epochs to print training stats

    Returns
    --------
        model (PyTorch model): trained cnn with best weights
        history (DataFrame): history of train and validation loss and accuracy
    """

    # Early stopping intialization
    epochs_no_improve = 0
    valid_loss_min = np.Inf

    valid_max_acc = 0
    history = []

    # Number of epochs already trained (if using loaded in model weights)
    try:
        print(f'Model has been trained for: {model.epochs} epochs.\n')
    except:
        model.epochs = 0
        print(f'Starting Training from Scratch.\n')

    overall_start = timer()

    # Main loop
    for epoch in range(n_epochs):

        # keep track of training and validation loss each epoch
        train_loss = 0.0
        valid_loss = 0.0

        train_acc = 0
        valid_acc = 0

        # Set to training
        model.train()
        start = timer()

        # Training loop
        for ii, (data, target) in enumerate(train_loader):
            # Tensors to gpu
            if torch.cuda.is_available():
                data, target = data.cuda(), target.cuda()

            
            
            # load autoencoder
            encoded,_ = autoencoder(data)
            # Clear gradients
            optimizer.zero_grad()
            # Predicted outputs are log probabilities
            output = model(encoded)

            # Loss and backpropagation of gradients
            loss = criterion(output, target)
            loss.backward()

            # Update the parameters
            optimizer.step()

            # Track train loss by multiplying average loss by number of examples in batch
            train_loss += loss.item() * data.size(0)

            # Calculate accuracy by finding max log probability
            _, pred = torch.max(output, dim=1)
            correct_tensor = pred.eq(target.data.view_as(pred))
            # Need to convert correct tensor from int to float to average
            accuracy = torch.mean(correct_tensor.type(torch.FloatTensor))
            # Multiply average accuracy times the number of examples in batch
            train_acc += accuracy.item() * data.size(0)

            # Track training progress
            print(
                f'Epoch: {epoch}\t{100 * (ii + 1) / len(train_loader):.2f}% complete. {timer() - start:.2f} seconds elapsed in epoch.',
                end='\r')

        # After training loops ends, start validation
        else:
            model.epochs += 1

            # Don't need to keep track of gradients
            with torch.no_grad():
                # Set to evaluation mode
                model.eval()

                # Validation loop
                for data, target in valid_loader:
                    # Tensors to gpu
                    if torch.cuda.is_available():
                        data, target = data.cuda(), target.cuda()
                    encoded,_ = autoencoder(data)
                    # Forward pass
                    output = model(encoded)

                    # Validation loss
                    loss = criterion(output, target)
                    # Multiply average loss times the number of examples in batch
                    valid_loss += loss.item() * data.size(0)

                    # Calculate validation accuracy
                    _, pred = torch.max(output, dim=1)
                    correct_tensor = pred.eq(target.data.view_as(pred))
                    accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
                    # Multiply average accuracy times the number of examples
                    valid_acc += accuracy.item() * data.size(0)

                # Calculate average losses
                train_loss = train_loss / len(train_loader.dataset)
                valid_loss = valid_loss / len(valid_loader.dataset)

                # Calculate average accuracy
                train_acc = train_acc / len(train_loader.dataset)
                valid_acc = valid_acc / len(valid_loader.dataset)

                history.append([train_loss, valid_loss, train_acc, valid_acc])

                # Print training and validation results
                if (epoch + 1) % print_every == 0:
                    print(
                        f'\nEpoch: {epoch} \tTraining Loss: {train_loss:.4f} \tValidation Loss: {valid_loss:.4f}'
                    )
                    print(
                        f'\t\tTraining Accuracy: {100 * train_acc:.2f}%\t Validation Accuracy: {100 * valid_acc:.2f}%'
                    )

                # Save the model if validation loss decreases
                if valid_loss < valid_loss_min:
                    # Save model
                    torch.save(model.state_dict(), save_file_name)
                    # Track improvement
                    epochs_no_improve = 0
                    valid_loss_min = valid_loss
                    valid_best_acc = valid_acc
                    best_epoch = epoch

                # Otherwise increment count of epochs with no improvement
                else:
                    epochs_no_improve += 1
                    # Trigger early stopping
                    if epochs_no_improve >= max_epochs_stop:
                        print(
                            f'\nEarly Stopping! Total epochs: {epoch}. Best epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
                        )
                        total_time = timer() - overall_start
                        print(
                            f'{total_time:.2f} total seconds elapsed. {total_time / (epoch+1):.2f} seconds per epoch.'
                        )

                        # Load the best state dict
                        model.load_state_dict(torch.load(save_file_name))
                        # Attach the optimizer
                        model.optimizer = optimizer

                        # Format history
                        history = pd.DataFrame(
                            history,
                            columns=[
                                'train_loss', 'valid_loss', 'train_acc',
                                'valid_acc'
                            ])
                        return model, history

    # Attach the optimizer
    model.optimizer = optimizer
    # Record overall time and print out stats
    total_time = timer() - overall_start
    print(
        f'\nBest epoch: {best_epoch} with loss: {valid_loss_min:.2f} and acc: {100 * valid_acc:.2f}%'
    )
    print(
        f'{total_time:.2f} total seconds elapsed. {total_time / (epoch):.2f} seconds per epoch.'
    )
    # Format history
    history = pd.DataFrame(
        history,
        columns=['train_loss', 'valid_loss', 'train_acc', 'valid_acc'])
    return model, history

In [0]:
model, history = train(
    model,
    autoencoder,
    criterion,
    optimizer,
    dataloaders['train'],
    dataloaders['val'],
    save_file_name='ensemble.pkl',
    max_epochs_stop=5,
    n_epochs=30,
    print_every=2)

Starting Training from Scratch.


Epoch: 1 	Training Loss: 1.4869 	Validation Loss: 1.4662
		Training Accuracy: 47.10%	 Validation Accuracy: 46.38%

Epoch: 3 	Training Loss: 1.3508 	Validation Loss: 1.4163
		Training Accuracy: 51.75%	 Validation Accuracy: 48.58%

Epoch: 5 	Training Loss: 1.2860 	Validation Loss: 1.4608
		Training Accuracy: 54.04%	 Validation Accuracy: 48.76%

Epoch: 7 	Training Loss: 1.2407 	Validation Loss: 1.4037
		Training Accuracy: 56.00%	 Validation Accuracy: 50.58%

Epoch: 9 	Training Loss: 1.2074 	Validation Loss: 1.3560
		Training Accuracy: 57.27%	 Validation Accuracy: 51.76%

Epoch: 11 	Training Loss: 1.1782 	Validation Loss: 1.3262
		Training Accuracy: 57.85%	 Validation Accuracy: 52.50%

Epoch: 13 	Training Loss: 1.1540 	Validation Loss: 1.2758
		Training Accuracy: 58.99%	 Validation Accuracy: 53.52%

Epoch: 15 	Training Loss: 1.1334 	Validation Loss: 1.2668
		Training Accuracy: 59.56%	 Validation Accuracy: 55.02%

Epoch: 17 	Training Loss: 1.1172 	Validatio

## (last)Function to Evaluate Model Over All Classes

The next function iterates through the testing set in order to make predictions for each image. It calculates performance for each category.

In [0]:
def accuracy(output, target, topk=(1, )):
    """Compute the topk accuracy(s)"""
    if torch.cuda.is_available():
        output = output.to('cuda')
        target = target.to('cuda')

    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        # Find the predicted classes and transpose
        _, pred = output.topk(k=maxk, dim=1, largest=True, sorted=True)
        pred = pred.t()

        # Determine predictions equal to the targets
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []

        # For each k, find the percentage of correct
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size).item())
        return res

In [0]:
def evaluate(model, autoencoder,test_loader, criterion, topk=(1, 5)):
    """Measure the performance of a trained PyTorch model

    Params
    --------
        model (PyTorch model): trained cnn for inference
        test_loader (PyTorch DataLoader): test dataloader
        topk (tuple of ints): accuracy to measure

    Returns
    --------
        results (DataFrame): results for each category

    """

    classes = []
    losses = []
    # Hold accuracy results
    acc_results = np.zeros((len(test_loader.dataset), len(topk)))
    i = 0

    model.eval()
    with torch.no_grad():

        # Testing loop
        for data, targets in test_loader:

            # Tensors to gpu
            if torch.cuda.is_available():
                data, targets = data.to('cuda'), targets.to('cuda')

            # Raw model output
            encoded,_ = autoencoder(data)
            out = model(encoded)
            # Iterate through each example
            for pred, true in zip(out, targets):
                # Find topk accuracy
                acc_results[i, :] = accuracy(
                    pred.unsqueeze(0), true.unsqueeze(0), topk)
                classes.append(model.idx_to_class[true.item()])
                # Calculate the loss
                loss = criterion(pred.view(1, n_classes), true.view(1))
                losses.append(loss.item())
                i += 1

    # Send results to a dataframe and calculate average across classes
    results = pd.DataFrame(acc_results, columns=[f'top{i}' for i in topk])
    results['class'] = classes
    results['loss'] = losses
    results = results.groupby(classes).mean()

    return results.reset_index().rename(columns={'index': 'class'})

In [0]:
criterion = nn.NLLLoss()
# Evaluate the model on all the training data
results = evaluate(model, autoencoder,dataloaders['test'], criterion)
results.head()